In [13]:
import pandas as pd
from sklearn_pandas import DataFrameMapper
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

df = pd.read_csv(r'Dataset(Advanced)(processed lyrics).csv') 

from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=10000,
                      lowercase=True,
                      ngram_range=(1,2),
                      analyzer = "word").fit(df['Lyrics'].values.astype(str))
print(len(bow.vocabulary_))
lyrics_bow = bow.transform(df['Lyrics'].values.astype(str))

tfidf_transformer = TfidfTransformer().fit(lyrics_bow)
lyrics_tfidf = tfidf_transformer.transform(lyrics_bow)

df['Bag Of Words'] = lyrics_tfidf

df['Lyrics'] = df['Lyrics'].astype(str)
mapper = DataFrameMapper([
     ('Lyrics', TfidfVectorizer()),
      ('Tempo', None),
      ('Energy', None),
       ('Loudness', None),
      ('Danceability', None),
      ('Speechiness', None),
      ('Acousticness', None),
     ('Artist Hit', None)
 ])

features = mapper.fit_transform(df[['Lyrics', 'Tempo', 'Energy', 'Loudness', 'Danceability', 'Speechiness'
                                    , 'Acousticness', 'Artist Hit']])
y = df['Hit']

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

model.fit(features, y)

10000


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
def predict():
    df['Lyrics_User'] = lyricsTextBox2.get(1.0,"end-1c")
    df['Lyrics_User'] = df['Lyrics_User'].values.astype(str)
    print(df['Lyrics_User'].head())
    print(type(df['Lyrics_User']))
    
    #Everything in lowercase
    df['Lyrics_User'] = df['Lyrics_User'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))

    #Removing punctuation that does not add meaning to the song
    df['Lyrics_User'] = df['Lyrics_User'].str.replace('[^\w\s]','')
    
    #Removing of stop words
    from nltk.corpus import stopwords

    stop = stopwords.words('english')
    df['Lyrics_User'] = df['Lyrics_User'].apply(lambda x: " ".join(x for x in str(x).split() if x not in stop))
    
    #Correction of Spelling mistakes
    from textblob import TextBlob
    df['Lyrics_User'] = df['Lyrics_User'].apply(lambda x: str(TextBlob(x).correct()))
    
    #Lemmatization is basically converting a word into its root word. It is preferred over Stemming.
    from textblob import Word
    df['Lyrics_User'] = df['Lyrics_User'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    
#     #CountVectorization of user lyrics
#     from sklearn.feature_extraction.text import CountVectorizer
#     user_bow = CountVectorizer(max_features=1000,
#                           lowercase=True,
#                           ngram_range=(1,1),
#                           analyzer = "word").fit([df['Lyrics_User']])
    
#     #Bag of Words conversion of user lyrics
#     user_lyrics_bow = bow.transform([df['Lyrics_User']])
    
#     #Tf-idf transforming of user lyrics
#     from sklearn.feature_extraction.text import TfidfTransformer
#     user_tfidf_transformer = TfidfTransformer().fit(user_lyrics_bow)
#     user_lyrics_tfidf = user_tfidf_transformer.transform(user_lyrics_bow)
    
    user_lyrics_bow = bow.transform(df['Lyrics_User'].values.astype(str))
    #df['Bag Of Words'] = lyrics_bow
    
    user_lyrics_tfidf = tfidf_transformer.transform(user_lyrics_bow)
    
    df['bog'] = user_lyrics_tfidf

    df['AP'] = float(ArtistPopularityEntry.get())
    df['SE'] = float(EnergyEntry.get())
    df['SL'] = float(LoudnessEntry.get())
    df['SA'] = float(AcousticnessEntry.get())
    df['ST'] = float(TempoEntry.get())
    df['SD'] = float(DanceabilityEntry.get())
    df['SS'] = float(SpeechinessEntry.get())
    
    mapper2 = DataFrameMapper([
     ('bog', None),
      ('ST', None),
      ('SE', None),
       ('SL', None),
      ('SD', None),
      ('SS', None),
      ('SA', None),
     ('AP', None)
    ])
    features2 = mapper2.fit_transform(df[['bog', 'ST', 'SE','SL','SD', 'SS', 'SA', 'AP']])
 

    user_prediction = model.predict(features2)
    print(user_prediction)
    if(user_prediction[0] == 1):
        resultLabel2.config(text = 'Song is Hit')
    else:
        resultLabel2.config(text = 'Song is not Hit')


In [15]:
import tkinter as tk
import tkinter.ttk as ttk

# creating root
root = tk.Tk()
root.geometry("800x500")
rows = 0

while rows < 2:
    root.rowconfigure(rows, weight=1)
    root.columnconfigure(rows, weight=1)
    rows += 1

# creating a frame for storing all widgets
AppFrame = tk.Frame(root)
AppFrame.grid(row=0, column=0)  # , columnspan=300)

# Creating label for title
AppTitle = tk.Label(AppFrame, text="Song HIT Prediction", font=("Arial", 30))
AppTitle.grid(row=0, column=0)
AppTitle2 = tk.Label(AppFrame, text="", font=("Arial", 5))
AppTitle2.grid(row=0, column=1)

''' Creating the first tab of the application... this tab is for the songwriter.'''
# Creating tabs for 2 use cases
notebook = ttk.Notebook(AppFrame)
notebook.grid(row=2, column=0, sticky="W", rowspan=100, columnspan=230)

# Define our first Tab. This tab contains textbox for
# entering lyrics and button for applying algorithm.
page1 = ttk.Frame(AppFrame)
notebook.add(page1, text='Lyrics Analysis')

# Label above textbox to tell user to "Enter lyrics Here".
EnterLyricsHereLabel = ttk.Label(page1, text="Enter Lyrics here:", padding=5)
EnterLyricsHereLabel.grid(row=1, column=0)

# Now create textbox in this "Lyrics Analysis" tab. Lyrics are input in this textbox
lyricsTextBox = tk.Text(page1, height=20, width=50)
lyricsTextBox.grid(row=2, column=0)  # , rowspan=50, columnspan=50)

# Button for Applying processing and prediction to Lyrics.
processButton = ttk.Button(page1, text="Apply Processing", width=40, command = predict)  # , command=predict)
processButton.grid(row=3, column=0)

# Label for showcasing the result.
resultLabel = ttk.Label(page1, text="\"Click Apply to see result\"",
                        font=("Arial", 20))
resultLabel.grid(row=5, column=0)

'''  This part of code now declares the page2, use case of music production team.'''

''' Creating page2 for Music Production company use cases features'''
page2 = ttk.Frame(AppFrame)
notebook.add(page2, text='Music Extras')

''' Adding Widgets on page2 Music Extras'''
EnterLyricsHereLabel2 = ttk.Label(page2, text="Enter Lyrics here:", padding=5)
EnterLyricsHereLabel2.grid(row=1, column=0)

# Now create textbox in this "Extras" tab. Lyrics are input in this textbox
lyricsTextBox2 = tk.Text(page2, height=10, width=40)
lyricsTextBox2.grid(row=2, column=0, columnspan=3)  # , rowspan=50, columnspan=50)

# Result Label
resultLabel2 = ttk.Label(page2, text="Enter details and click \"Apply\" \nto see result", font=("Arial", 10), padding=7)
resultLabel2.grid(row=2, column=3)

# Artist popularity entry box
ArtistPopularityLabel = ttk.Label(page2, text="Artist popularity: ", padding=8)
ArtistPopularityLabel.grid(row=3, column=0, sticky='E')
ArtistPopularityEntry = ttk.Entry(page2)
ArtistPopularityEntry.grid(row=3, column=1, sticky="W")

# Song Tempo label and entry
TempoLabel = ttk.Label(page2, text="Song Tempo: ")
TempoLabel.grid(row=3, column=3, sticky="E")
TempoEntry = ttk.Entry(page2)
TempoEntry.grid(row=3, column=4, sticky="W")

# Energy label and entry
EnergyLabel = ttk.Label(page2, text="Song Energy: ")
EnergyLabel.grid(row=4, column=0, sticky="E")
EnergyEntry = ttk.Entry(page2)
EnergyEntry.grid(row=4, column=1, sticky="W")

# Danceability label and entry
DanceabilityLabel = ttk.Label(page2, text="Song Danceability")
DanceabilityLabel.grid(row=4, column=3, sticky="E")
DanceabilityEntry = ttk.Entry(page2)
DanceabilityEntry.grid(row=4, column=4, sticky="W")

# Loudness label and entry Spotify measures loudness by ReplayGain standard.
LoudnessLabel = ttk.Label(page2, text="Song Loudness", padding=8)
LoudnessLabel.grid(row=5, column=0, sticky="E")
LoudnessEntry = ttk.Entry(page2)
LoudnessEntry.grid(row=5, column=1, sticky="W")

# Speechiness label and entry
SpeechinessLabel = ttk.Label(page2, text="Song Speechiness: ")
SpeechinessLabel.grid(row=5, column=3, sticky="E")
SpeechinessEntry = ttk.Entry(page2)
SpeechinessEntry.grid(row=5, column=4, sticky="W")

# Acousticness label and entry
AcousticnessLabel = ttk.Label(page2, text="Song Acousticness", padding=8)
AcousticnessLabel.grid(row=6, column=0, sticky="E")
AcousticnessEntry = ttk.Entry(page2)
AcousticnessEntry.grid(row=6, column=1, sticky="W")

# Button for Applying processing and prediction to Lyrics.
processButton2 = ttk.Button(page2, text="Apply Processing", width=40, padding=10,  command=predict)
processButton2.grid(row=7, column=0, columnspan=8)

# main loop the root window
root.mainloop()

0    hey there
1    hey there
2    hey there
3    hey there
4    hey there
Name: Lyrics_User, dtype: object
<class 'pandas.core.series.Series'>


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\moksh\Anaconda3\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-14-e788a506c182>", line 70, in predict
    user_prediction = model.predict(features2)
  File "C:\Users\moksh\Anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 66, in predict
    jll = self._joint_log_likelihood(X)
  File "C:\Users\moksh\Anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 724, in _joint_log_likelihood
    X = check_array(X, accept_sparse='csr')
  File "C:\Users\moksh\Anaconda3\lib\site-packages\sklearn\utils\validation.py", line 433, in check_array
    array = np.array(array, dtype=dtype, order=order, copy=copy)
ValueError: setting an array element with a sequence.
